In [61]:
import numpy as np
import sympy as sp
import math as m
import matplotlib.pyplot as plt

In [62]:
#puntos = puntos a  grafifcar
#f = funcion a graficar 
def grafico(puntos_x,puntos_y,f):
    PUNTO_X_MAX =  max(puntos_x)
    PUNTO_X_MIN = min(puntos_x)
    DOMINIO = 5
    #genero datos para el dominio de x
    x_values = np.arange(PUNTO_X_MIN-DOMINIO,DOMINIO+PUNTO_X_MAX,step=0.3)
    #genero datos para la imagen de y
    y_values = ([f(x) for x in x_values])
    #ploteo el grafico
    plt.plot(x_values,y_values)
    # Agregar el eje de abscisas
    plt.axhline(y=0, color='black', linestyle='--')
    # Agregar el eje de ordenadas
    plt.axvline(x=0, color='black', linestyle='--')
    #agrego puntos calulados 
    plt.scatter(puntos_x,puntos_y,c="red")
    #agrego labels
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.title(f'Grafico aproximado de la funcion')
    #muestro el grafico
    plt.show()

In [63]:
def decimal_truncate(numero, decimales):
    if (decimales < 0): raise Exception("el N° de decimales a truncar debe ser positivo")
    factor = 10.0 ** decimales
    return m.trunc(numero * factor) / factor


In [64]:
#funcion que ingresa los decimales exactos
def decExactos_a_tolerancia(exactitud):
    if (exactitud < 0): raise Exception("La exactitud debe ser positiva o 0")
    #scamos el valor de paradad dada la exactitud
    tolerancia = "0."
    #agregamos tantos 0 como exactitud indicada
    for i in range(exactitud): tolerancia += "0"
    #agregamos un uno para indicar que es una fraccion
    tolerancia += "1"
    #convertimos a flotante para poder operar
    return float(tolerancia)
"""
    x: recibe un valor cercano a la raiz que se busca aproximar
    tolerancia: 0 < Numero < 1 que indica el intervalo de corte de la funcion ej: 0.0001 (como minimo3 deciamles exactos)
    f: funcion de tipo numerica
"""
def aproximar_raiz_newton_secante(x,tolerancia,f):
    symbol = sp.symbols("x")
    #instanciamos derivada de la funcion
    f1 = sp.lambdify(symbol,sp.diff(f(symbol)),np)
    #anidamos funcion para poder utilizar variables no locales
    def secante():
        try:
            return x1 - ((f(x1)*(x1-x))/(f(x1)-f(x)))
        except ZeroDivisionError:
            print(f"error: f({x}) = 0, no se puede dividir por 0")
            exit(6)
    #------------------------- fin secante ----------------------------------    
    while(True):    
        try:
            x1 = x - (f(x)/f1(x))
        except ZeroDivisionError:
            print(f"error: f'({x}) = 0, no se puede dividir por 0")
            exit(6)
        if (abs(x-x1) <= tolerancia):
            break
        x = secante()
    return (x1)

In [65]:
def productoria(points, step, txt=False, **kwargs):
    x =  kwargs['x'] if 'x' in kwargs else points[step][0]
    evaluado = 1
    noEvaluado = ""
    for i in range(step):
        noEvaluado += f'*(x-{points[i][0]})'
        evaluado *= (x - points[i][0])
    return evaluado if not txt else noEvaluado   

In [66]:
def alphas(points:list):
    alphaList = [] 
    
    def prevAlphas(p):
        acum = 0
        for i in range(len(alphaList)):
            acum += alphaList[i] if i < 1 else alphaList[i]*productoria(points,i,x=p)
        return acum

    for i,p in enumerate(points):           
        alpha = (points[i][1]-(prevAlphas(p[0]) if i>0 else 0))/productoria(points, i) 
        alphaList.append(alpha)
       
    return alphaList

In [67]:
def nEquation(points, **kwargs):
    pn =  kwargs['pn'] if 'pn' in kwargs else len(points)
    x = sp.symbols('x')
    equations = []
    alphaList = alphas(points)
    equation = f'{alphaList[0]}'

    for i in range(pn):
        if i > 0:
            equation += f'{"+" if alphaList[i] > 0 else ""}{alphaList[i]}{productoria(points,i,True)}'
    equation = sp.expand(equation)
    return sp.lambdify(x,equation,np)

In [68]:
def validarNum(txt:str, nat=False, **kwargs):
    min =  kwargs['min'] if 'min' in kwargs else None
    while True:
        val = input(txt)  
        try:
            val = float(val)
            if val.is_integer():
                val = int(val)
            if (min and val < min):
                raise Exception(f"Error: El número debe ser mayor de {min}") 
            if (nat and (val <= 0)):
                raise Exception("Error: Se esperaba un número mayor a 0")                
            return val
        except ValueError:
            print("Error: Se esperaba un valor numérico válido.") 
        except Exception as e:
            print(e)             

In [69]:
def choice(txt:str):
    ch = str(input(txt))
    while(ch.lower() != 's' and ch.lower() != 'n'):
        print(f'Se esperaba S/N pero se obtubo {ch}.')
        ch = str(input(txt))
    return True if ch.lower() == 's' else False

In [70]:
class Puntos:
    def __init__(self, manualStep=False):
        self.steps = int(validarNum('Ingrese la cantidad de puntos:',True)) if manualStep else 20
        self.XN = np.array([])
        self.YN = np.array([])
    
    def generar(self):
        for i in range(self.steps):
            self.XN = np.append(self.XN, validarNum(f'Ingrese el valor para X{i}'))
            self.YN = np.append(self.YN, validarNum(f'Ingrese el valor para Y{i}'))

    def obtenerPuntos(self):
        self.generar()
        return [(x,y) for x,y in zip(self.XN, self.YN)]

class PuntosAleatorios(Puntos):
    def __init__(self, X_Equal_Y, manualStep=False):
        Puntos.__init__(self, manualStep)
        self.x_min = validarNum('Ingrese la cota menor para x:')
        self.x_max = validarNum('Ingrese la cota mayor para x:',min=self.x_min)
        self.y_min = self.x_min
        self.y_max = self.x_max
    
        if not X_Equal_Y:
            self.y_min = validarNum('Ingrese la cota inferior para y:')
            self.y_max = validarNum('Ingrese la cota superior para y:',min=self.y_min)
         
    def generar(self):
        self.XN = np.random.uniform(self.x_min, self.x_max, self.steps)
        self.YN = np.random.uniform(self.y_min, self.y_max, self.steps)

In [74]:
#cantPuntos = cantidad de puntos que se usaron para generar el polinomio
#funcNewton = polinomio de newton numerico
def testeoGrado(cantPuntos:int,FuncNewton):
    GRADO_TEORICO = cantPuntos-1
    x = sp.symbols("x")
    f_simbolic = sp.simplify(FuncNewton(x))
    GRADO_POLINOMIO = sp.degree(f_simbolic)
    if (GRADO_POLINOMIO != GRADO_TEORICO): raise Exception(f"error: el grado del polinomio ({GRADO_POLINOMIO}) debe ser igual al grado teorico ({GRADO_TEORICO})")
    print(f"El grado del polinomio ({GRADO_POLINOMIO}) es igual al grado teorico (cantPuntos-1 = {GRADO_TEORICO}) ")
    

In [77]:
def saltosDeLinea(cantSaltos):
    for i in range(cantSaltos): print("\n")
def main():
    ##ingreso de datos
    try:
        lista_puntos = Puntos().obtenerPuntos() if choice("Desea ingresar los puntos manualmente? S/N") else PuntosAleatorios(choice('Limites de x e y iguales? S/N')).obtenerPuntos()
        lista_puntos_asc = sorted(lista_puntos, key=lambda x: x[0])
        lista_puntos_desc = sorted(lista_puntos, key=lambda x: x[0], reverse=True)
        saltosDeLinea(2)
        print("puntos generados ordenados segun x de forma decreciente:")
        print(lista_puntos_desc)
    except Exception:
        print('La creación de la lista de puntos falló.')
    ##calulo de polinomio generado por puntos ordenados segun x de forma de decreciente
    saltosDeLinea(2)
    poli_newton_desc = nEquation(lista_puntos_desc)
    print("Polinomio generado:")
    print(sp.simplify(poli_newton_desc(x)))
    ##Testeo de grado del polinomio ordenado por x
    saltosDeLinea(1)
    testeoGrado(cantPuntos=len(lista_puntos_desc),FuncNewton=poli_newton_desc)
main()
    

Se esperaba S/N pero se obtubo .




puntos generados ordenados segun x de forma decreciente:
[(7.815907819063187, 6.081085183951728), (7.584505457641255, 4.266541286203074), (7.18755585978499, 7.005838627898713), (6.831265919585726, 5.1390006968796085), (6.637775163633705, 3.7501819520455943), (6.5511636209293345, 4.948575249768205), (6.322051307370523, 7.374064393500587), (5.615227506030532, 7.798213169060807), (5.327960231298148, 4.648009609832228), (5.107015260590105, 2.266564873474757), (4.7776908897040355, 5.133134302297465), (4.106495852255209, 2.9153196438803617), (4.088563023710032, 2.50292031222182), (4.004677435116705, 1.5457381614475496), (3.2973945868994754, 2.207525573545049), (2.7666471219752076, 5.740234279321949), (2.225812570204499, 2.4250084791699122), (1.9939331816427233, 3.5397541701430786), (1.4212522812627204, 6.891578167664101), (1.3580324472715064, 1.0161031530166946)]




Polinomio generado:
-0.00134121251906771*x**19 + 0.121688838932628*x**18 - 5.177197566619